In [1]:
import pandas as pd
import os

In [ ]:

def gerar_totais (arquivo_csv_cadastro):

    display (f'Processando arquivo: {arquivo_csv_cadastro}')
    
    indice_ultimo_separador = arquivo_csv_cadastro.rfind("/")
    nome_arquivo = arquivo_csv_cadastro [indice_ultimo_separador+1:-4] 

    ano = int(nome_arquivo[:4])
    mes = int(nome_arquivo[4:6])        
    
    df_cadastro = pd.read_csv(arquivo_csv_cadastro, encoding='iso-8859-1', sep=';')

    df_contagem_combinacoes = df_cadastro.groupby(['ORGSUP_EXERCICIO', 'ORG_EXERCICIO', 'UORG_EXERCICIO', 'ORGSUP_LOTACAO', 'ORG_LOTACAO', 'UORG_LOTACAO', 'REGIME_JURIDICO', 'DESCRICAO_CARGO']).size().reset_index(name='Contagem')    

    df_contagem_combinacoes['ano'] = ano
    df_contagem_combinacoes['mes'] = mes
    
    return df_contagem_combinacoes
    
    

In [ ]:

TAMANHO_BLOCO = 12

def percorre_diretorio_CSV(diretorio, destino_csv):

    #Declarando variáveis globais para o escopo da função percorre_diretorio_CSV
    #elas serão usadas na função salvar_e_reiniciar_parte
    global numero_parte
    global contador_arquivos_processados    
    global df_consolidado

    
    #Variáveis para calcular tamanho da parte    
    numero_parte = 0        
    contador_arquivos_processados = 0

    
    #Dataframe para consolidação
    df_consolidado = pd.DataFrame()

    
    
    def salvar_e_reiniciar_parte():

        #Referenciando as variáveis globais criadas pela função pai percorre_diretorio_CSV
        global numero_parte
        global contador_arquivos_processados
        global df_consolidado        
        
        numero_parte += 1
            
        #colocar número da parte no nome do CSV consolidado       
        nome_destino_csv = f'{destino_csv[:-4]}_parte_{numero_parte}{destino_csv[-4:]}'

        display (f'*********************** Salvando arquivo de parte: {nome_destino_csv}')
        
        #Salva o bloco atual
        df_consolidado.to_csv(nome_destino_csv, index=False)
        
        #Reinicia o Dataframe
        df_consolidado = pd.DataFrame()

        contador_arquivos_processados = 0

    
    
    # Lista todos os arquivos no diretório
    arquivos = os.listdir(diretorio)    
    
    # Filtra apenas os subdiretorios, excluindo arquivos
    subdiretorios = [arquivo for arquivo in arquivos if os.path.isdir(os.path.join(diretorio, arquivo))]
    
    # Ordena os subdiretórios alfabeticamente
    subdiretorios_ordenados = sorted(subdiretorios)
    
    
    # Itera a lista de subdiretorios
    for subdiretorio in subdiretorios_ordenados:

        contador_arquivos_processados += 1
        
        caminho_completo = os.path.join(diretorio, subdiretorio)
        
        arquivos_subdiretorio = os.listdir(caminho_completo) 

        lista_arquivo_cadastro = [arquivo for arquivo in arquivos_subdiretorio if arquivo.lower().find("cadastro") != -1]

        if len(lista_arquivo_cadastro) == 1:
            
            arquivo_cadastro = lista_arquivo_cadastro[0]

            caminho_completo_arquivo_cadastro = os.path.join(caminho_completo, arquivo_cadastro)                        

            df_total_mes = gerar_totais (caminho_completo_arquivo_cadastro)

            # Concatena o DataFrame atual com o DataFrame consolidado
            df_consolidado = pd.concat([df_consolidado, df_total_mes], ignore_index=True)
        
        else:

            display (f'ATENÇÃO: Mais ou nenhum arquivo cadastro no subdiretório {caminho_completo}')

        
        #Se formou uma parte com TAMANHO_BLOCO arquivos a serem salvos
        if contador_arquivos_processados >= TAMANHO_BLOCO:

            salvar_e_reiniciar_parte()

    
    #Se existe um resto de arquivos que ainda não foram salvos em uma parte
    if contador_arquivos_processados > 0:

            salvar_e_reiniciar_parte()
    
        

In [ ]:
percorre_diretorio_CSV("../../dados/dados_siape_bruto/descompactado", "../../dados/cadastro_siape_consolidado_orgao_mes_ano.csv")

In [2]:
def percorre_diretorio_partes(diretorio, destino_csv):
    
    #Dataframe para consolidação
    df_consolidado_total= pd.DataFrame()

    # Lista todos os arquivos no diretório    
    arquivos_partes = os.listdir(diretorio) 
    arquivos_partes = [arquivo for arquivo in arquivos_partes if arquivo.lower().find("parte") != -1]

    # Itera a lista de arquivos
    for arquivo_parte in arquivos_partes:
                
        caminho_completo_arquivo_parte = os.path.join(diretorio, arquivo_parte) 

        display (f'Processando arquivo: {arquivo_parte}')
        
        #df_consolidado = pd.read_csv(caminho_completo_arquivo_parte, encoding='iso-8859-1', sep=';')
        df_consolidado = pd.read_csv(caminho_completo_arquivo_parte)

        # Concatena o DataFrame atual com o DataFrame consolidado
        df_consolidado_total = pd.concat([df_consolidado_total, df_consolidado], ignore_index=True)
                

    display (f'Vai salvar....')

    #Salva o arquivo consolidado com todos os anos
    df_consolidado_total.to_csv(destino_csv, index=False)


    display (f'Finalizado!')

In [ ]:
percorre_diretorio_partes("../../dados", "../../dados/cadastro_siape_consolidado_completo.csv")

'Processando arquivo: cadastro_siape_consolidado_orgao_mes_ano_parte_11.csv'

'Processando arquivo: cadastro_siape_consolidado_orgao_mes_ano_parte_5.csv'

'Processando arquivo: cadastro_siape_consolidado_orgao_mes_ano_parte_2.csv'

'Processando arquivo: cadastro_siape_consolidado_orgao_mes_ano_parte_4.csv'

'Processando arquivo: cadastro_siape_consolidado_orgao_mes_ano_parte_8.csv'

...

'Processando arquivo: cadastro_siape_consolidado_orgao_mes_ano_parte_3.csv'

'Processando arquivo: cadastro_siape_consolidado_orgao_mes_ano_parte_10.csv'

'Processando arquivo: cadastro_siape_consolidado_orgao_mes_ano_parte_9.csv'

'Processando arquivo: cadastro_siape_consolidado_orgao_mes_ano_parte_1.csv'

'Vai salvar....'